In [1]:
import sys
sys.path.insert(0, '../')
import numpy as np
%reload_ext autoreload
%autoreload 1
%aimport allel
%aimport allel.model.ndarray
allel.__version__

'0.22.0.dev0'

## GenotypeVector

In [2]:
g = allel.model.ndarray.GenotypeVector([[0, 1], [-1, -1]]*20)
g.is_phased = np.array([True, False]*20, dtype=bool)
g

0,1,2,3,4,...,35,36,37,38,39
0|1,./.,0|1,./.,0|1,...,./.,0|1,./.,0|1,./.


In [3]:
g.display(30, 10)

0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,...,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.


In [4]:
g.displayall()

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.


In [5]:
print(repr(g))

GenotypeVector((40, 2), dtype=int64)
0|1 ./. 0|1 ./. 0|1  ...  ./. 0|1 ./. 0|1 ./.


In [6]:
print(g)

0|1 ./. 0|1 ./. 0|1  ...  ./. 0|1 ./. 0|1 ./.


In [7]:
print(g.to_str(40))

0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./.


## GenotypeArray

In [8]:
g = allel.model.ndarray.GenotypeArray([[[0, 1], [-1, -1]]*20]*20)
g.is_phased = np.array([[True, False, False, False]*10]*20, dtype=bool)
g.mask = np.array([[1, 0, 0, 0, 0]*8]*20, dtype=bool)

In [9]:
print(g)

.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
...
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.



In [10]:
print(repr(g))

GenotypeArray((20, 40, 2), dtype=int64)
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
...
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.



In [12]:
g

GenotypeArray((20, 40, 2), dtype=int64)
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
...
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.

In [86]:
g.display(row_threshold=40, col_threshold=40)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
0,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
1,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
2,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
3,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
4,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
5,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
6,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
7,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
8,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
9,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.


In [87]:
g[0]

0,1,2,3,4,...,35,36,37,38,39
.|.,./.,0/1,./.,0|1,...,./.,0|1,./.,0/1,./.


In [88]:
g[:, 4]

0,1,2,3,4,...,15,16,17,18,19
0|1,0|1,0|1,0|1,0|1,...,0|1,0|1,0|1,0|1,0|1


In [89]:
g[:, 4].displayall()

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1


In [90]:
g.mask = None
g

GenotypeArray((20, 40, 2), dtype=int64)
0|1 ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
0|1 ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
0|1 ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
...
0|1 ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
0|1 ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
0|1 ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.

In [91]:
g.displayall()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
0,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
1,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
2,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
3,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
4,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
5,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
6,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
7,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
8,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
9,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.


In [92]:
h = g.to_haplotypes()

In [93]:
type(h)

allel.model.ndarray.HaplotypeArray

In [94]:
h

HaplotypeArray((20, 80), dtype=int64)
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
...
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .

In [95]:
print(repr(h))

HaplotypeArray((20, 80), dtype=int64)
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
...
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .



In [96]:
print(h)

0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
...
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .



In [97]:
h.display(20, 40)

,0,1,2,3,4,...,75,76,77,78,79
0,0,1,.,.,0,...,.,0,1,.,.
1,0,1,.,.,0,...,.,0,1,.,.
2,0,1,.,.,0,...,.,0,1,.,.
3,0,1,.,.,0,...,.,0,1,.,.
4,0,1,.,.,0,...,.,0,1,.,.
5,0,1,.,.,0,...,.,0,1,.,.
6,0,1,.,.,0,...,.,0,1,.,.
7,0,1,.,.,0,...,.,0,1,.,.
8,0,1,.,.,0,...,.,0,1,.,.
9,0,1,.,.,0,...,.,0,1,.,.


In [98]:
h.displayall()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79
0,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.
1,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.
2,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.
3,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.
4,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.
5,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.
6,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.
7,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.
8,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.
9,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.


In [99]:
ac = g.count_alleles()
ac

AlleleCountsArray((20, 2), dtype=int32)
20 20
20 20
20 20
...
20 20
20 20
20 20

In [100]:
ac.displayall()

,0,1
0,20,20
1,20,20
2,20,20
3,20,20
4,20,20
5,20,20
6,20,20
7,20,20
8,20,20
9,20,20


In [101]:
print(ac)

20 20
20 20
20 20
...
20 20
20 20
20 20



In [102]:
print(repr(ac))

AlleleCountsArray((20, 2), dtype=int32)
20 20
20 20
20 20
...
20 20
20 20
20 20



In [103]:
ac + 5

AlleleCountsArray((20, 2), dtype=int32)
25 25
25 25
25 25
...
25 25
25 25
25 25

In [104]:
ac + ac

AlleleCountsArray((20, 2), dtype=int32)
40 40
40 40
40 40
...
40 40
40 40
40 40

In [105]:
ac.vstack(ac)

array([[20, 20],
       [20, 20],
       [20, 20],
       [20, 20],
       [20, 20],
       [20, 20],
       [20, 20],
       [20, 20],
       [20, 20],
       [20, 20],
       [20, 20],
       [20, 20],
       [20, 20],
       [20, 20],
       [20, 20],
       [20, 20],
       [20, 20],
       [20, 20],
       [20, 20],
       [20, 20],
       [20, 20],
       [20, 20],
       [20, 20],
       [20, 20],
       [20, 20],
       [20, 20],
       [20, 20],
       [20, 20],
       [20, 20],
       [20, 20],
       [20, 20],
       [20, 20],
       [20, 20],
       [20, 20],
       [20, 20],
       [20, 20],
       [20, 20],
       [20, 20],
       [20, 20],
       [20, 20]], dtype=int32)

## Legacy

In [2]:
# setup an array of genotype calls
shape = n_variants, n_samples, ploidy = 50000, 1000, 2
data = np.zeros(shape, dtype='i1')
# simulate some mutations
n_alleles = n_variants * n_samples * ploidy
idx = np.random.randint(0, (n_alleles//2)-1, size=n_alleles//20)
data[:, :, 1].reshape((-1))[idx] = 1
data[:, :, 0].reshape((-1))[idx[:n_alleles//200]] = 1
g = allel.GenotypeArray(data, copy=False)

In [3]:
g

,0,1,2,3,4,...,995,996,997,998,999
0,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
1,0/0,0/0,0/0,0/0,0/0,...,0/0,1/1,0/0,0/0,0/0
2,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
3,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
4,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,1/1,0/0


In [4]:
g.take([0, 1, 2], axis=1)

,0,1,2
0,0/0,0/0,0/0
1,0/0,0/0,0/0
2,0/0,0/0,0/0
3,0/0,0/0,0/0
4,0/0,0/0,0/0


In [5]:
h = g.to_haplotypes()
h

,0,1,2,3,4,...,1995,1996,1997,1998,1999
0,0,0,0,0,0,...,0,0,0,0,0
1,0,0,0,0,0,...,0,0,0,0,0
2,0,0,0,0,0,...,0,0,0,0,0
3,0,0,0,0,0,...,0,0,0,0,0
4,0,0,0,0,0,...,0,1,1,0,0


In [6]:
h.take([0, 1, 2], axis=1)

,0,1,2
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0


In [7]:
gc = allel.GenotypeChunkedArray(g).copy()
gc

,0,1,2,3,4,...,995,996,997,998,999
0,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
1,0/0,0/0,0/0,0/0,0/0,...,0/0,1/1,0/0,0/0,0/0
2,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
3,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
4,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,1/1,0/0


In [8]:
gc.take([0, 1, 2], axis=1)

,0,1,2
0,0/0,0/0,0/0
1,0/0,0/0,0/0
2,0/0,0/0,0/0
3,0/0,0/0,0/0
4,0/0,0/0,0/0


In [9]:
hc = gc.to_haplotypes()
hc

,0,1,2,3,4,...,1995,1996,1997,1998,1999
0,0,0,0,0,0,...,0,0,0,0,0
1,0,0,0,0,0,...,0,0,0,0,0
2,0,0,0,0,0,...,0,0,0,0,0
3,0,0,0,0,0,...,0,0,0,0,0
4,0,0,0,0,0,...,0,1,1,0,0


In [10]:
hc.take([0, 1, 2], axis=1)

,0,1,2
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0


In [11]:
ac = g.count_alleles()
ac

,0,1
0,1911,89
1,1895,105
2,1896,104
3,1892,108
4,1892,108


In [12]:
acc = gc.count_alleles()
acc

,0,1
0,1911,89
1,1895,105
2,1896,104
3,1892,108
4,1892,108


In [13]:
gc.is_called()

carray((50000, 1000), bool)
  nbytes: 47.68 MB; cbytes: 743.19 KB; ratio: 65.70
  cparams := cparams(clevel=5, shuffle=True, cname='blosclz')
[[ True  True  True ...,  True  True  True]
 [ True  True  True ...,  True  True  True]
 [ True  True  True ...,  True  True  True]
 ..., 
 [ True  True  True ...,  True  True  True]
 [ True  True  True ...,  True  True  True]
 [ True  True  True ...,  True  True  True]]

In [14]:
gc.to_n_alt()

ChunkedArray((50000, 1000), int8, nbytes=47.7M, cbytes=13.0M, cratio=3.7, cname=blosclz, clevel=5, shuffle=True, chunks=(524, 1000), data=bcolz.carray_ext.carray)

In [15]:
pos = np.unique(np.random.randint(0, 1000000, size=50000))
pos

array([     9,     28,     30, ..., 999946, 999958, 999960])

In [16]:
ref = np.random.choice([b'A', b'C', b'T', b'G'], size=pos.shape[0])
ref

array([b'A', b'T', b'A', ..., b'G', b'G', b'A'], 
      dtype='|S1')

In [17]:
alt = np.random.choice([b'A', b'C', b'T', b'G'], size=pos.shape[0])
alt

array([b'G', b'A', b'A', ..., b'A', b'T', b'A'], 
      dtype='|S1')

In [18]:
ra = np.rec.fromarrays([pos, ref, alt], names=['POS', 'REF', 'ALT'])
vtbl = allel.VariantTable(ra)
vtbl

POS,REF,ALT
9,b'A',b'G'
28,b'T',b'A'
30,b'A',b'A'
43,b'C',b'T'
53,b'G',b'A'


In [19]:
vctbl = allel.VariantChunkedTable(vtbl).copy()
vctbl

POS,REF,ALT
9,b'A',b'G'
28,b'T',b'A'
30,b'A',b'A'
43,b'C',b'T'
53,b'G',b'A'


In [20]:
vtbl[:5].display(10)

POS,REF,ALT
9,b'A',b'G'
28,b'T',b'A'
30,b'A',b'A'
43,b'C',b'T'
53,b'G',b'A'


In [21]:
vtbl[:5].display(None)

POS,REF,ALT
9,b'A',b'G'
28,b'T',b'A'
30,b'A',b'A'
43,b'C',b'T'
53,b'G',b'A'


In [22]:
vtbl[:5].displayall()

POS,REF,ALT
9,b'A',b'G'
28,b'T',b'A'
30,b'A',b'A'
43,b'C',b'T'
53,b'G',b'A'


In [23]:
vctbl.take(list(range(5))).display(10)

POS,REF,ALT
9,b'A',b'G'
28,b'T',b'A'
30,b'A',b'A'
43,b'C',b'T'
53,b'G',b'A'


In [24]:
vctbl.take(list(range(5))).display(None)

POS,REF,ALT
9,b'A',b'G'
28,b'T',b'A'
30,b'A',b'A'
43,b'C',b'T'
53,b'G',b'A'


In [25]:
vctbl.take(list(range(5))).displayall()

POS,REF,ALT
9,b'A',b'G'
28,b'T',b'A'
30,b'A',b'A'
43,b'C',b'T'
53,b'G',b'A'
